### Обработка данных 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint

In [2]:
def caesar_cipher(text, k):
    encrypted_text = ""
    for s in text:
        order = new_order = ord(s)
        if s.isalpha():
            new_order = order + k
            if s.isupper():
                if new_order > ord("Я"):
                    new_order = ord('А') + order + k - ord('Я') - 1
            else:
                if new_order > ord("я"):
                    new_order = ord('а') + order + k - ord('я') - 1
        encrypted_text += chr(new_order)
    return encrypted_text

In [3]:
CHARS = list('абвгдежзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [4]:
# все строки списками из индексов
def raw_to_indexes(lines):
    indexes = torch.zeros((len(lines), MAX_LEN), dtype=int)
    for i in range(len(lines)):
        if raw_lines[i]:
            for j, w in enumerate(lines[i]):
                if j >= MAX_LEN: break
                indexes[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    return indexes

In [5]:
# обучающие данные
MAX_LEN = 40

# все строки
with open('datasets/onegin.txt', 'r', encoding='utf-8') as file:
    raw_lines = [line.strip().lower() for line in file.readlines()]
    
# все зашифрованные строки
encrypted_raw_lines = [caesar_cipher(i, randint(1,20)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

### Полученные тензоры данных

In [6]:
indexes

tensor([[ 0, 13, 15,  ...,  0,  0,  0],
        [11, 15,  4,  ...,  0,  0,  0],
        [15, 14, 33,  ...,  0,  0,  0],
        ...,
        [ 3, 33,  9,  ...,  0,  0,  0],
        [24, 29,  6,  ...,  0,  0,  0],
        [ 9, 33,  8,  ...,  0,  0,  0]])

In [7]:
encrypted_indexes

tensor([[ 0, 18, 20,  ...,  0,  0,  0],
        [17, 21, 10,  ...,  0,  0,  0],
        [23, 22, 33,  ...,  0,  0,  0],
        ...,
        [19, 33, 25,  ...,  0,  0,  0],
        [12, 17, 26,  ...,  0,  0,  0],
        [23, 33, 22,  ...,  0,  0,  0]])

In [8]:
indexes.shape, encrypted_indexes.shape # 1452 строки, длиной 40 символов

(torch.Size([1452, 40]), torch.Size([1452, 40]))

### Создание и обучение модели 

In [9]:
class Decryptor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decryptor, self).__init__()
        self.embedding = nn.Embedding(input_size, 64)
        self.rnn = nn.RNN(64, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.rnn(embedded)
        output = self.linear(output)
        return output

In [10]:
input_size = len(CHAR_TO_INDEX)
hidden_size = 128
output_size = len(CHAR_TO_INDEX)

In [11]:
model = Decryptor(input_size, hidden_size, output_size)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
num_epochs = 200
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(indexes)
    loss = criterion(output.view(-1, output_size), encrypted_indexes.view(-1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 10, Loss: 2.0776243209838867
Epoch: 20, Loss: 1.812370777130127
Epoch: 30, Loss: 1.7154252529144287
Epoch: 40, Loss: 1.6705760955810547
Epoch: 50, Loss: 1.6380863189697266
Epoch: 60, Loss: 1.612605094909668
Epoch: 70, Loss: 1.590772032737732
Epoch: 80, Loss: 1.5721725225448608
Epoch: 90, Loss: 1.5561192035675049
Epoch: 100, Loss: 1.5422699451446533
Epoch: 110, Loss: 1.5302631855010986
Epoch: 120, Loss: 1.5198222398757935
Epoch: 130, Loss: 1.5107576847076416
Epoch: 140, Loss: 1.5028926134109497
Epoch: 150, Loss: 1.4960488080978394
Epoch: 160, Loss: 1.4900583028793335
Epoch: 170, Loss: 1.4847722053527832
Epoch: 180, Loss: 1.4800699949264526
Epoch: 190, Loss: 1.4758579730987549
Epoch: 200, Loss: 1.472062349319458


### Тестирование данных 

In [14]:
with torch.no_grad():
    output = model(indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)

In [15]:
decrypted_strings

['яьх кдкд хдаие ехешуяе ьцдимп',
 'рфжшд ул р вчцры сдпорфж',
 'ьы ырдумще хжзд пдтбдимп',
 'м пщеьн илищрдае хл рфж',
 'лзф хцшрлц кцьнор цдьрд',
 'бь зпмл рфх ыдодл хрьрд',
 'я гпскхяр хржлще м клуе м сфэа',
 'бо фщеэид ум ьднщ ьцтэк',
 'рдофх умпофл рфийцтжиь',
 'вфсщцмиьжф сдмдмпеще',
 'лры ьфжщбрч ытхцйипдще',
 'вхэфпехь хтжсфхшае пуодцтжиь',
 'искзедще м кщрдае хцт хжзд',
 'рфжшд кл жжцш дтпкрлц блзд',
 '',
 'щдч кщрдп рчптжфп хфрлтд',
 'яуцд и хбсм уд утэцэдйе',
 'ищжилвбоп зщпун плилтд',
 'бйхсузсмч зщже хрьмы цтисяе',
 'ицщпел яткумпб о цщесдуй',
 'я жуцтжр рчхзф цтшдуй',
 'зпс уцхимчстимп ыжн кл жфх',
 'вфсмтпецл хтссйтфрмае идх',
 'ьыозоу шфмцбн рфх хцшлщзск',
 'цтимпте уд зцлзде улил',
 'ззз яфулц ззще цтимпмхе ил',
 'мпм зпмхщдпм хфх юмаданпк',
 'щда улофжшд зщпгщ м г',
 'бь рчжилу хжилц ксе рлсе ',
 '',
 'ясэцми тщсмжфэ зпдмфцтисф',
 'ифпздрм кмп ззф фшфв',
 'идийп ццм здпй йллзфжуф',
 'м ыцтшфадпте удчфэов',
 'яьздид йилуумл ецжпмпд',
 'яхжцрй  сд умр ыщимпд',
 'вфцфш  